<a href="https://colab.research.google.com/github/ndsoi/ndsoi/blob/main/MyFun_CubicSpline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$h_i = x_{i+1}-x_i$$
$$\alpha_i = \frac{h_{i-1}}{h_{i-1}+h_i}$$
$$\beta_i = 3[\frac{1-\alpha_i}{h_{i-1}}(y_i-y_{i-1})+\frac{\alpha_i}{h_i}(y_{i+1}-y_i)]$$

In [ ]:
def calhi(X):
  H = []
  # len(X) = n+1
  for i in range(len(X)-1):
    H.append(X[i+1]-X[i])
  return H

def calalphai(H):
  # len(H) n  0——n-1
  # 只当 A0 = 0
  A = [0]
  for i in range(1,len(H)):
    A.append((H[i-1]/(H[i-1]+H[i])))
  return A

def calbetai(A,H,Y):
  # len(A) = n 实际有效的alpha在[1:]范围内
  # Y n+1  0——n
  B = [0]
  for i in range(1,len(A)):
    tmp1 = (1-A[i])/H[i-1]*(Y[i]-Y[i-1])
    tmp2 = A[i]/H[i]*(Y[i+1]-Y[i])
    B.append(3*(tmp1+tmp2))
  return B



补充的数学表达式
$$
2m_0+m_1 = \frac{3}{h_0}(y_1-y_0)
$$

$$
m_{n-1}+2m_n = 3\frac{y_n-y_{n-1}}{h_{n-1}}
$$

In [ ]:
# 构造相应的矩阵
# 先构造补充条件的list
import numpy as np

def RowForSupplement(n):
  RowsFS = np.zeros(shape=(2,n+1))
  RowsFS[0][0] = 2
  RowsFS[0][1] = 1
  RowsFS[1][n-1] = 1
  RowsFS[1][n] = 2
  return RowsFS

# 构造原先的n-1个条件的方程
def Row(A):
  n = len(A)
  Rows = np.zeros(shape=(n-1,n+1))
  for i in range(1,n):
    Rows[i-1][i-1] = 1-A[i]
    Rows[i-1][i] = 2
    Rows[i-1][i+1] = A[i]
  return Rows

# 合并左侧矩阵
def Concat(Rows,RowsFS,n):
  C = np.zeros(shape=(n+1,n+1))
  C[:n-1,:] = Rows
  C[n-1:,:] = RowsFS
  return C

# 构造β矩阵
def Beta(B,Y,H):
  n = len(Y)-1
  l1 = 3*(Y[1]-Y[0])/H[0]
  l2 = 3*(Y[n]-Y[n-1])/H[n-1]
  Beta = np.zeros(shape=(n+1,))
  for i in range(1,n):
    Beta[i-1] = B[i]
  Beta[n-1] = l1
  Beta[n] = l2
  return Beta

# 解线性方程
def solve(X,Y):
  H = calhi(X)
  A = calalphai(H)
  B = calbetai(A,H,Y)
  RowsFS = RowForSupplement(len(Y)-1)
  Rows = Row(A)
  Left = Concat(Rows,RowsFS,len(Y)-1)
  beta = Beta(B,Y,H)
  M = np.linalg.solve(Left,beta)
  print(M)
  return M







[ 2.125  1.75  -1.25  -2.375]


array([ 2.125,  1.75 , -1.25 , -2.375])

In [ ]:
# 预测代码
def predict(targetX,X,Y):
  M = solve(X,Y)
  n = len(Y)
  # 确定targetX所在的区间
  st = 0
  for i in range(n-1):
    if targetX >= X[i] and targetX <= X[i+1]:
      st = i
      break
  tmp1 = (1+2*(targetX-X[st])/(X[st+1]-X[st]))*pow(((targetX-X[st+1])/(X[st]-X[st+1])),2)
  tmp2 = (1+2*(targetX-X[st+1])/(X[st]-X[st+1]))*pow(((targetX-X[st])/(X[st+1]-X[st])),2)

  tmp3 = (targetX-X[st])*pow(((targetX-X[st+1])/(X[st]-X[st+1])),2)
  tmp4 = (targetX-X[st+1])*pow(((targetX-X[st])/(X[st+1]-X[st])),2)

  return tmp1*Y[st]+tmp2*Y[st+1]+tmp3*M[st]+tmp4*M[st+1]


In [ ]:
X = [1,2,4,5]
Y = [1,3,4,2]
solve(X,Y)

# 验证正确
predict(3,X,Y)

[ 2.125  1.75  -1.25  -2.375]
[ 2.125  1.75  -1.25  -2.375]


4.25

In [ ]:
# 再做一组测试
X = [75,76,77,78,79,80]
Y = [2.768,2.833,2.903,2.979,3.062,3.153]

re = predict(78.3,X,Y)
print(f"x=78.3的预测值是{re}")

[0.06399522 0.06700957 0.07296651 0.0791244  0.08753589 0.09273206]
x=78.3的预测值是3.003044526315789


库内函数

In [ ]:

from scipy.interpolate import CubicSpline


x = np.array(X)
y = np.array(Y)

# 使用 CubicSpline 进行三次样条插值
cs = CubicSpline(x, y)

# 生成插值点
targetX = np.array([78.3])
targetY = cs(targetX)
print(f"target={targetY}")




target=[3.0031195]
